<a href="https://colab.research.google.com/github/medha-hegde/master_thesis/blob/main/thesis_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Clone Repo
import os

if not os.path.exists("master_thesis"):
  !git clone https://github.com/medha-hegde/master_thesis.git

In [ ]:
#@title Install Dependencies and Restart Runtime
%cd /content/
!pip install -r master_thesis/requirements.txt -qq

import os
os.kill(os.getpid(), 9)

In [ ]:
#@title Run Preliminary Experiment

%cd '/content/master_thesis/preliminary experiments'
# !cp '/content/master_thesis/preliminary experiments/prelim_exp.py' .
# !cp '/content/master_thesis/preliminary experiments/prelim_helpers.py' .

exp_name = "experiment 1" #@param ["experiment 1", "experiment 2", "experiment 3", "experiment 4"] {allow-input: true}

from prelim_exp import run_prelim_exp
run_prelim_exp(exp_name)

In [ ]:
#@title Run Character Probe Experiment
model_card = "openai/clip-vit-large-patch14" #@param ["openai/clip-vit-base-patch32", "openai/clip-vit-large-patch14", "t5-base", "t5-large"] {allow-input: true}
# device = "gpu"

%cd '/content/master_thesis/character-probe'

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

# Replace model name in params file
# Read in the file
with open('params.py', 'r') as file :
  filedata = file.read()

# Replace the target string

for model_name in ["openai/clip-vit-base-patch32", "openai/clip-vit-large-patch16", "t5-base", "t5-large"]:
  if model_name in filedata:
      filedata = filedata.replace( model_name,model_card)

# Write the file out again
with open('params.py', 'w') as file:
  file.write(filedata)

!python3 train.py 

## Main Experiments

In [ ]:
text_encoder_name = "openai/clip-vit-large-patch14" #@param ["openai/clip-vit-base-patch32", "openai/clip-vit-large-patch14", "t5-small", "t5-base","google/byt5-small","google/byt5-base"] {allow-input: true}
epochs = 150 #@param {type:"number"}
combine_with_text_encoder = "None" #@param ["None","openai/clip-vit-base-patch32", "openai/clip-vit-large-patch14", "t5-small", "t5-base","google/byt5-small","google/byt5-base"] {allow-input: true}

# W&B Configs

import wandb
config = {
  "pretrained_model_name_or_path" : text_encoder_name,
   "pretrained_model_name_or_path_2" : combine_with_text_encoder,
    "textmodel_maxtokens" : 77,
    "text_prompt" : "a black and white image of the word",
    "img_size" : 64,
    "sample_batch_size" : 64,
    "dummy_run": False,
    "batch_size": 128,
    "model_name" :"transformer",
    "model_save_path" : "/content/",
    "checkpoint" : None,
    "lr" : 10e-4,
    "epochs" : epochs,
    "device" : "cuda"

}
config["run_name"] = config["pretrained_model_name_or_path"].replace("/","_")


%cd '/content/master_thesis'

from main_experiments.create_imgs import create_imgs
from main_experiments.text_model import load_text_model
from main_experiments.create_torch_dataset import create_torch_dataset
from main_experiments.unet_setup import UNet_SD, marginal_prob_std_fn, get_n_params
import torch

# Create Training Image-Text Dataset
create_imgs(config)

# Load Text Model + tokenizer
tokenizer, text_encoder = load_text_model(config["pretrained_model_name_or_path"])
config["text_emb_length"]  = list(text_encoder.named_parameters())[0][1].shape[1]
config["text_emb_length_2"]  = 0

if combine_with_text_encoder != "None":
  tokenizer_2, text_encoder_2 = load_text_model(config["pretrained_model_name_or_path_2"])
  config["text_emb_length_2"]  = list(text_encoder_2.named_parameters())[0][1].shape[1]


# Create torch dataset
if combine_with_text_encoder == "None":
  train_dataloader = create_torch_dataset(config, tokenizer)
else:
  train_dataloader = create_torch_dataset(config, tokenizer, tokenizer_2)



# Load U-Net model
device = "cuda"
context_dim = config["text_emb_length"] + config["text_emb_length_2"] 

score_model = torch.nn.DataParallel(UNet_SD(marginal_prob_std=marginal_prob_std_fn,context_dim=context_dim))
score_model = score_model.to(device)
config["no_of_params"] = get_n_params(score_model)

# Run Training 

if config["checkpoint"] is not None:
  checkpoint = torch.load(config["checkpoint"], map_location=device)
else:
  checkpoint = None


wandb.init(project="text-encoder-experiments_testing",name = config["run_name"])
wandb.config.update(config)

from main_experiments.train import train_diffusion_model
  
train_diffusion_model(config,
                      train_dataloader,
                      score_model,
                      tokenizer,
                      text_encoder,
                      tokenizer_2=tokenizer_2 if combine_with_text_encoder != "None" else None,
                      text_encoder_2=text_encoder_2 if combine_with_text_encoder != "None" else None,
                      n_epochs =  config["epochs"],
                      lr=config["lr"],
                      model_name=config["model_name"],
                      checkpoint = checkpoint,
                      model_save_path=config["model_save_path"],
                      dummy_run=config["dummy_run"])

wandb.finish()

In [ ]:
#@title Run OCR
model_name = "CLIP_small" #@param 
model_saved_path = "/content/ckpt_transformer_clip_t5.pt" #@param 
text_encoder_name = "openai/clip-vit-base-patch32" #@param ["openai/clip-vit-base-patch32", "openai/clip-vit-large-patch14", "t5-small", "t5-base","google/byt5-small","google/byt5-base"] {allow-input: true}
combine_with_text_encoder = "None" #@param ["None","openai/clip-vit-base-patch32", "openai/clip-vit-large-patch14", "t5-small", "t5-base","google/byt5-small","google/byt5-base"] {allow-input: true}


ocr_configs = {"model_name" : model_name,
              "model_card" : text_encoder_name,
               "pretrained_model_name_or_path_2" : combine_with_text_encoder,
                "model_saved_path" : model_saved_path,
              "textmodel_maxtokens" : 77,
              "device" : "cuda"}

%cd '/content/master_thesis'
from main_experiments.ocr import run_ocr

run_ocr(ocr_configs)

In [ ]:
#@title Plot OCR Results
model_1_name =  'CLIP' #@param 
model_2_name = 'T5' #@param 
model_3_name = 'ByT5' #@param 

model_1_saved_path = "small_ocr_resultsCLIP.json" #@param 
model_2_saved_path = "small_ocr_resultsT5.json" #@param 
model_3_saved_path = "small_ocr_resultsByT5.json" #@param 

ocr_plot_configs = {"model_names" : [model_1_name,model_2_name,model_3_name],
                    "model_paths" : [model_1_saved_path,model_2_saved_path,model_3_saved_path]
}

%cd '/content/master_thesis'
from main_experiments.ocr import ocr_plot

ocr_plot(ocr_plot_configs)
